In [2]:
import pickle
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

from scipy import stats

from scipy import sparse
from copy import deepcopy

In [3]:
EPS = 1e-6

# 1
Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:  
   * взять в тренировочный набор турниры с dateStart из 2019 года;   
   * в тестовый — турниры с dateStart из 2020 года.


In [4]:
with open('chgk/players.pkl', 'rb') as f:
    players = pickle.load(f)
    
with open('chgk/results.pkl', 'rb') as f:
    results = pickle.load(f)
    
with open('chgk/tournaments.pkl', 'rb') as f:
    tournaments = pickle.load(f)

Сформируем обучающую и тестовую выборки

In [5]:
train_tournaments = {}
test_tournaments = {}
for tour_id, value in tournaments.items():
    if value['dateStart'][: 4] == '2019':
        train_tournaments[tour_id] = value
    elif value['dateStart'][: 4] == '2020':
        test_tournaments[tour_id] = value

Выберем только те турниры, в которых есть повопросные результаты (уберем из рассмотрения турниры, где есть незачтенные вопросы).

In [6]:
results_with_mask = {}
for tour_id, teams in results.items():
    if teams:
        teams_with_mask = 0
        len_masks = []
        for team in teams:
            if 'mask' in team and team['mask'] is not None and '?' not in team['mask'] \
            and 'X' not in team['mask']:
                teams_with_mask += 1
                len_masks.append(len(team['mask']))
        if teams_with_mask == len(teams) and max(len_masks) == min(len_masks):
            results_with_mask[tour_id] = teams

In [7]:
len(results_with_mask)

3178

Теперь нам надо выбрать турниры, которые есть и в `train_tournaments`, и в `results_with_mask`. Аналогично для тестового набора данных

In [8]:
train_tournaments_with_mask = {}
test_tournaments_with_mask = {}

train_resuls_with_mask = {}
test_resuls_with_mask = {}
for tour_id, teams in results_with_mask.items():
    if tour_id in train_tournaments:
        train_tournaments_with_mask[tour_id] = train_tournaments[tour_id]
        train_resuls_with_mask[tour_id] = teams
    elif tour_id in test_tournaments:
        test_tournaments_with_mask[tour_id] = test_tournaments[tour_id]
        test_resuls_with_mask[tour_id] = teams

In [9]:
print(f'Число турниров в обучающей выборке: {len(train_tournaments_with_mask)}')
print(f'Число турниров в тестовой выборке: {len(test_tournaments_with_mask)}')

Число турниров в обучающей выборке: 591
Число турниров в тестовой выборке: 149


# 2
Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:  
   * повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;  
   * в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;  
   * для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков


In [10]:
def get_info_from_tournament(id_tour, teams):
    '''Извлекает id команды, id игроков и повопросную маску'''
    list_teams = []
    list_players = []
    list_mask = []
    for team in teams:
        list_teams.append(team['team']['id'])
        list_mask.append(team['mask'])
        players = []
        for player in team['teamMembers']:
            players.append(player['player']['id'])
        list_players.append(players)
    return list_teams, list_players, list_mask

In [11]:
def get_id_question(id_tour, mask):
    '''Присваивает каждому вопросу уникальный id вида <id_tour>_<num_question>'''
    id_question = []
    for i in range(len(mask)):
        id_question.append(str(id_tour) + '_' + str(i))
    return id_question

In [12]:
def get_results_for_team(id_tour, id_team, players, mask, id_questions, is_train=True, train_players=None):
    results = []
    if is_train:
        for id_player in players:
            for id_quest, answer in zip(id_questions, mask):
                #print(mask)
                results.append((id_tour, id_team, id_player, id_quest, int(answer)))
    else:
        for id_player in players:
            # оставляем только тех игроков, которые есть в обучающей выборке
            if id_player in train_players:
                # добавляем фиктивный id вопроса, т.к. мы не можем сделать повопросное предсказание
                results.append((id_tour, id_team, id_player, '0', sum(map(int, mask)), len(mask)))
    return results

In [13]:
def get_results_for_data(data_dict, is_train=True, train_players=None):
    all_results = []
    for id_tour, teams in data_dict.items():
        list_teams, list_players, list_mask = get_info_from_tournament(id_tour, teams)
        id_questions = get_id_question(id_tour, list_mask[0])
        for id_team, players, mask in zip(list_teams, list_players, list_mask):
            res = get_results_for_team(id_tour, id_team, players, mask, id_questions, is_train, train_players)
            all_results.extend(res)
    return all_results

In [14]:
train_results = get_results_for_data(train_resuls_with_mask)

In [15]:
train_df = pd.DataFrame(train_results, columns=['id_tournament', 'id_team', 'id_player', 'id_question', 'is_answer'])

In [16]:
train_df.head(10)

,id_tournament,id_team,id_player,id_question,is_answer
0,4772,45556,6212,4772_0,1
1,4772,45556,6212,4772_1,1
2,4772,45556,6212,4772_2,1
3,4772,45556,6212,4772_3,1
4,4772,45556,6212,4772_4,1
5,4772,45556,6212,4772_5,1
6,4772,45556,6212,4772_6,1
7,4772,45556,6212,4772_7,1
8,4772,45556,6212,4772_8,1
9,4772,45556,6212,4772_9,0


In [17]:
ohe = OneHotEncoder(handle_unknown='ignore')

X_train = ohe.fit_transform(train_df[['id_player', 'id_question']])
y_train = train_df['is_answer']

In [18]:
X_train.shape

(13454728, 81725)

In [19]:
lr = LogisticRegression(random_state=71)

lr.fit(X_train, y_train)

/home/tgorlenko/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=71)

Будем считать, что модель сопоставила каждому игроку вес, который равен его вкладу (чем больше коэффициент, тем большее влияние оказывает игрок).

In [20]:
# сначала идут игроки, затем вопросы
lr.coef_.shape

(1, 81725)

In [21]:
list_players = ohe.categories_[0]
list_questions = ohe.categories_[1]

In [22]:
power_players = lr.coef_[0, : len(list_players)]

Для удобства создадим словарь соответствия id игрока и его ФИО

In [23]:
def id_player_to_name(players_data):
    id2name_dict = {}
    for id_player, info in players_data.items():
        id2name_dict[id_player] = info['surname'] + ' ' + info['name']
    return id2name_dict

In [24]:
id2name_dict = id_player_to_name(players)

In [25]:
ranging_players_df = pd.DataFrame({'id_player': list_players, 'power': power_players})
ranging_players_df['player_name'] = ranging_players_df['id_player'].map(id2name_dict)
ranging_players_df.sort_values(by='power', inplace=True, ascending=False)

ranging_players_df.head(10)

,id_player,power,player_name
3746,27403,3.870750,Руссо Максим
583,4270,3.840984,Брутер Александра
4115,30152,3.675417,Сорожкин Артём
3816,27822,3.622571,Савченков Михаил
3932,28751,3.615617,Семушин Иван
4130,30260,3.590023,Спектор Евгений
4135,30270,3.514355,Спешков Сергей
3132,22799,3.380658,Николенко Сергей
3570,26089,3.378090,Прокофьева Ирина
9031,87637,3.372710,Саксонов Антон


Если сравнивать результат модели с рейтингом на [сайте](https://rating.chgk.info/players.php?release=1557&page=1&order=rating), то видим, что модель выдает адекватные результаты.

# 3

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
 * предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
 * в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

Создадим тестовую выборку. Так как мы не можем сделать предсказание по для игроков, которых не было в обучающей выборке, то уберем их из тестовой выборки. Еще в новом турнире нам неизвестны вопросы, т.е. информацию о них мы не сможем использовать, в столбцах с `id_question` поставим фиктивный ноль. Будем делать предсказания только на основе силы игрока.  
Будем считать, что вероятность команды верно ответить на вопрос (сила команды) равна вероятности того, что хотя бы один из игроков верно ответил на вопрос:
$P(team = 1) = 1 - П_{i} (1 - P(player_i = 1))$

Истинную силу команды будем считать как доля правильных ответов в турнире.

По силе команды (предсказанной или истинной) можно отранжировать команды.

In [26]:
test_results = get_results_for_data(test_resuls_with_mask, is_train=False, train_players=list_players)

In [27]:
test_df = pd.DataFrame(test_results, columns=['id_tournament', 'id_team', 'id_player', 'id_question', 'sum_answer', 'count_questions'])

In [28]:
X_test = test_df[['id_tournament', 'id_team', 'id_player', 'id_question']]

In [29]:
X_test.head()

,id_tournament,id_team,id_player,id_question
0,5414,66120,18490,0
1,5414,66120,116901,0
2,5414,66120,8532,0
3,5414,66120,42346,0
4,5414,66120,123190,0


In [30]:
X_test = ohe.transform(X_test[['id_player', 'id_question']])

In [31]:
pred_proba = lr.predict_proba(X_test)

In [32]:
# вероятность того, что игрок верно ответит на вопрос
pred_proba = pred_proba.T[1]

In [33]:
def print_metrics(data_df, preds):
    data = deepcopy(data_df)
    data['pred_proba'] = preds
    data['score_pred'] = data.groupby(['id_tournament', 'id_team'])['pred_proba'].transform(lambda x: 1 - np.prod(1 - x))
    help_df = data[['id_tournament', 'id_team', 'sum_answer', 'count_questions', 'score_pred']].drop_duplicates().reset_index(drop=True)
    help_df['score_real'] = help_df['sum_answer'] / help_df['count_questions']
    
    print(f"Корреляция Спирмана: {help_df.groupby('id_tournament').apply(lambda x: stats.spearmanr(x['score_real'], x['score_pred']).correlation).mean()}")
    print(f"Корреляция Кендалла: {help_df.groupby('id_tournament').apply(lambda x: stats.kendalltau(x['score_real'], x['score_pred']).correlation).mean()}")

In [34]:
print_metrics(test_df, pred_proba)

Корреляция Спирмана: 0.7907965680970711
Корреляция Кендалла: 0.6325955740915766


Корреляции в указанном диапазоне.

# 4
Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
* предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
* разработайте EM-схему для обучения этой модели, реализуйте её в коде;
* обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


Раньше мы полагали, что если команда верно ответила на вопрос, то каждый игрок команды верно ответил на вопрос. В реальности это не так.  
Будем считать, что если хотя бы один игрок ответил на вопрос верно, то и команда ответила на вопрос верно, т.е.  
$P(team = 0| player = 1) = 0$ и $P(team = 1| player = 1) = 1$.

Вероятность команды ответить на вопрос верно будем считать так же, как и раньше:

$P(team = 1) = 1 - П_{i}P(player_i = 0) = 1 - П_{i} (1 - P(player_i = 1))$  
Попробуем оценить вероятность того, что игрок ответил на вопрос верно, при условии, что команда ответила на вопрос верно:  

$P(player = 1|team = 1) = \dfrac{P(team = 1| player = 1) \cdot P(player = 1)}{P(team = 1)} = \dfrac{1 \cdot P(player = 1)}{P(team = 1)} = \dfrac{P(player = 1)}{P(team = 1)} = \dfrac{P(player = 1)}{1 - П_{i} (1 - P(player_i = 1))}$

На Е-шаге будем оценивать скрытую переменную $P(player = 1|team = 1)$. На М-шаге решаем задачу оптимизации, где целевая переменная -- это наша скрытая переменная. Вероятность предсказания -- это $P(player = 1)$

In [35]:
from sklearn.preprocessing import MinMaxScaler

In [36]:
def E_step(data, preds):
    help_df = data.copy()
    help_df['pred_proba'] = preds
    help_df['score_pred'] = help_df.groupby(['id_tournament', 'id_team'])['pred_proba'].transform(lambda x: 1 - np.prod(1 - x))
    y = help_df['pred_proba'] / help_df['score_pred']
    y = y.values.reshape(-1, 1)
    scaler = MinMaxScaler()
    scaler.fit(y)
    y = scaler.transform(y)
    y[help_df['is_answer'] == 0] = 0
    y = y.flatten()
    return y

In [37]:
def log_loss(y_true, y_pred):
    return - np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [38]:
def predict(X, w):
    return sigmoid(X.dot(w))

In [39]:
def M_step(X, y, w, learnig_rate=15, epochs=100):
    min_loss = np.inf
    for epoch in range(epochs):
        prediction = predict(X, w)
        grad = X.T.dot(prediction - y) / len(y)
        w -= learnig_rate * grad
        cur_loss = log_loss(y, prediction)
        if abs(min_loss - cur_loss) < EPS:
            break
        min_loss = cur_loss

In [40]:
def EM_algo(X_train_ohe, train_dataframe, X_test_ohe, test_dataframe, w='baseline', n_iter=10, learnig_rate=15, epochs=100):
    '''ЕМ алгоритм
    :param X_train_ohe: матрица признаков с ohe
    :param train_dataframe: dataframe с тренировочными данными
    :param X_test_ohe: матрица признаков с ohe
    :param test_dataframe: dataframe с тестовыми данными
    :param w: начальная инициализация весов для логрегрессии
    :param n_iter: число итераций в ЕМ-алгоритме
    :param learnig_rate: скорость обучения
    :param epochs: число эпох для логрегрессии на М-шаге
    
    :return w: обученные веса модели
    '''
    X_tr = deepcopy(X_train_ohe)
    X_tr = sparse.hstack((np.ones((X_tr.shape[0], 1)), X_tr), format='csr')
    X_te = deepcopy(X_test_ohe)
    X_te = sparse.hstack((np.ones((X_te.shape[0], 1)), X_te), format='csr')
    if w == 'baseline':
        w = np.hstack([lr.intercept_, lr.coef_[0]])
    else:
        # случайная инициализация
        w = np.random.randn(X_tr.shape[1])
    print('Start:')
    print_metrics(test_dataframe, predict(X_te, w))
    for i in range(n_iter): 
        print()
        print(f'Iter {i}:')
        preds = predict(X_tr, w)
        y = E_step(train_dataframe, preds)
        M_step(X_tr, y, w, learnig_rate, epochs)
        print_metrics(test_dataframe, predict(X_te, w))
    return w

In [41]:
em_w = EM_algo(X_train, train_df, X_test, test_df, n_iter=10)

Start:
Корреляция Спирмана: 0.7907965680970711
Корреляция Кендалла: 0.6325955740915766

Iter 0:
Корреляция Спирмана: 0.795582681676043
Корреляция Кендалла: 0.6378873250938423

Iter 1:
Корреляция Спирмана: 0.7973454450570451
Корреляция Кендалла: 0.6398208170615309

Iter 2:
Корреляция Спирмана: 0.7980690583322327
Корреляция Кендалла: 0.6409667986430179

Iter 3:
Корреляция Спирмана: 0.7979463348574457
Корреляция Кендалла: 0.6408386726138786

Iter 4:
Корреляция Спирмана: 0.798045920520992
Корреляция Кендалла: 0.6409993475826181

Iter 5:
Корреляция Спирмана: 0.7979718813866471
Корреляция Кендалла: 0.6410239043836757

Iter 6:
Корреляция Спирмана: 0.7980231726459638
Корреляция Кендалла: 0.6411056005563086

Iter 7:
Корреляция Спирмана: 0.7980230944229496
Корреляция Кендалла: 0.6411565057313605

Iter 8:
Корреляция Спирмана: 0.7980909668207663
Корреляция Кендалла: 0.6411984834570531

Iter 9:
Корреляция Спирмана: 0.7980940926581909
Корреляция Кендалла: 0.6411907003830308


Видим, что метрики немного растут

# 5
А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? *Если будет интересно:* постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

In [42]:
id2name_tournament = {}
for id_tour in pd.unique(train_df['id_tournament']):
    id2name_tournament[id_tour] = tournaments[id_tour]['name']

In [43]:
id2weight_question = dict(zip(list_questions, em_w[-len(list_questions):]))

In [44]:
ranging_tournament_df = train_df.loc[:, ['id_tournament', 'id_question']]
ranging_tournament_df['weight_question'] = ranging_tournament_df['id_question'].map(id2weight_question)
ranging_tournament_df['name_tournament'] = ranging_tournament_df['id_tournament'].map(id2name_tournament)

ranging_tournament_df.head(10)

,id_tournament,id_question,weight_question,name_tournament
0,4772,4772_0,2.748722,Синхрон северных стран. Зимний выпуск
1,4772,4772_1,1.499639,Синхрон северных стран. Зимний выпуск
2,4772,4772_2,0.038934,Синхрон северных стран. Зимний выпуск
3,4772,4772_3,0.432184,Синхрон северных стран. Зимний выпуск
4,4772,4772_4,2.698555,Синхрон северных стран. Зимний выпуск
5,4772,4772_5,1.998776,Синхрон северных стран. Зимний выпуск
6,4772,4772_6,1.966689,Синхрон северных стран. Зимний выпуск
7,4772,4772_7,-1.239945,Синхрон северных стран. Зимний выпуск
8,4772,4772_8,-1.174167,Синхрон северных стран. Зимний выпуск
9,4772,4772_9,-3.207460,Синхрон северных стран. Зимний выпуск


In [45]:
ranging_tournament_df = ranging_tournament_df.groupby(['id_tournament', 'name_tournament'])['weight_question'].mean().reset_index()
ranging_tournament_df.sort_values(by='weight_question', inplace=True)
ranging_tournament_df.rename(columns={'weight_question': 'ranging_tournament'}, inplace=True)

Топ-10 сложных турниров

In [46]:
ranging_tournament_df.head(10)

,id_tournament,name_tournament,ranging_tournament
583,6149,Чемпионат Санкт-Петербурга. Первая лига,-4.294826
473,5928,Угрюмый Ёрш,-2.407325
35,5159,Первенство правого полушария,-2.093407
559,6101,Воображаемый музей,-2.068810
246,5587,Записки охотника,-1.831467
334,5693,Знание – Сила VI,-1.761344
12,5025,Кубок городов,-1.681896
22,5083,Ускользающая сова,-1.680473
484,5942,Чемпионат Мира. Этап 2. Группа В,-1.599843
197,5515,Чемпионат Минска. Лига А. Тур четвёртый,-1.597542


Топ-10 простых турниров

In [47]:
ranging_tournament_df.tail(10)

,id_tournament,name_tournament,ranging_tournament
148,5457,Студенческий чемпионат Калининградской области,1.581954
353,5729,Синхрон-lite. Выпуск XXX,1.694118
342,5704,(а)Синхрон-lite. Лига старта. Эпизод X,1.749687
8,5011,(а)Синхрон-lite. Лига старта. Эпизод IV,1.780352
61,5313,(а)Синхрон-lite. Лига старта. Эпизод VI,1.891151
337,5698,(а)Синхрон-lite. Лига старта. Эпизод VII,1.899452
525,6003,Второй тематический турнир имени Джоуи Триббиани,1.915207
341,5702,(а)Синхрон-lite. Лига старта. Эпизод IX,2.021746
6,5009,(а)Синхрон-lite. Лига старта. Эпизод III,2.045876
10,5013,(а)Синхрон-lite. Лига старта. Эпизод V,2.144869


Судя по названиям турниров, ранжирование адекватное